# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import json

# Import API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Create city dataframe
city_data = pd.read_csv('output_files/city_data.csv')
city_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mar del plata,-38.0023,-57.5575,59.00,71,90,4.61,AR,1616052184
1,1,nome,64.5011,-165.4064,3.20,72,1,6.91,US,1616052185
2,2,bredasdorp,-34.5322,20.0403,69.80,68,6,6.91,ZA,1616052186
3,3,seoul,37.5683,126.9778,66.20,39,75,11.50,KR,1616052187
4,4,cayenne,4.9333,-52.3333,78.01,94,100,9.15,GF,1616052188


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = city_data[["Lat", "Lng"]]

# Store humidity values into a list for weigh
humidity = city_data["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=4)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather = city_data[(city_data["Max Temp"] <80) & (city_data["Max Temp"] >70) & (city_data["Humidity"] <70) & (city_data["Wind Speed"] <10) & (city_data["Cloudiness"] ==0)]
ideal_weather

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,maltahohe,-24.8333,16.9833,73.02,16,0,4.68,NaN,1616052198
60,60,new norfolk,-42.7826,147.0587,75.99,41,0,1.99,AU,1616052231
97,97,port lincoln,-34.7333,135.8667,74.75,57,0,9.66,AU,1616052258
288,288,atar,20.5169,-13.0499,70.86,13,0,8.63,MR,1616052401
295,295,vryburg,-26.9566,24.7284,72.66,58,0,7.72,ZA,1616052406
433,433,ardakan,32.3100,54.0175,78.80,24,0,9.22,IR,1616052507


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Store ideal_weather data into hotel_df and add the hotel name column
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,17,maltahohe,-24.8333,16.9833,73.02,16,0,4.68,NaN,1616052198,
1,60,new norfolk,-42.7826,147.0587,75.99,41,0,1.99,AU,1616052231,
2,97,port lincoln,-34.7333,135.8667,74.75,57,0,9.66,AU,1616052258,
3,288,atar,20.5169,-13.0499,70.86,13,0,8.63,MR,1616052401,
4,295,vryburg,-26.9566,24.7284,72.66,58,0,7.72,ZA,1616052406,
5,433,ardakan,32.3100,54.0175,78.80,24,0,9.22,IR,1616052507,


In [7]:
# Store geocoordinates
target_radius = 50000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # run a request using our params dictionary and convert to json
    hotel_name = requests.get(base_url, params=params).json()
    
    # pprint the json
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][1]["name"]
    except:
        print("Missing field/result. Skipping...")
        

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIxFthaTVqEX1n--7ZxRAlFYXUPOlFmezltAk33bxJWjTcGXffryi6PGdR-3VmwADh-7qPt2KrddBLrngSH4OiC-u6DpGdQngIzim9xHd7qx6IjZUbv0kqg1A7Ug2W_ppFOc3RfYoGxbgflCUDHWyswwSCX0aNKQwW3EZP5IrfREFBD44aTS0trY3bTTRNs6KyJtbCjV6AO5yAD3I-nsv40kwU_1j_k5gsW6ypa7zN6cKLeVjzylKXDEGFKwNhNbkG9CzJoH1yUAD6tmpR7MXc4p679_W00Lm1unLm_P-Gk9StSUMttkqkG_xYn7zGzfUPCZsrCgYOqZrookKlfijjjxdLiF3v95bGwQeqFyAlHcVcKsyq5hrJR3sjNIhShnQwCTwwfLp0HYrrMEyLrdXgcQliV-0wNtg_LK_V9o5SIel9Td_xCag",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -24.8380587,
                    "lng": 16.9801346
                },
                "viewport": {
                    "northeast": {
                        "lat": -24.82849444207751,
                        "lng": 16.99657926099018
                    },
                    "southwest": {
                        "lat": -24.85341907703714,
                        "lng": 16.965637267089

{
    "html_attributions": [],
    "next_page_token": "ATtYBwL6xCCZc1B2HlhQIK-1S6cDVLKivuR0rEbO8XeM9yCsbYWn-wumZRNQuSnNiJw5Lh0FgqJRMdWd8pdpBoJE18XpRTaBVGr_uenMXsAPhKyJgGjFTK3cJhEi2S7D64tzup7tUZWRIczL1WB1SpKQesJjFSASXKZ5H1oZiOJx-ulsct-I_ZZVSiz20TDgOpWLxWTRcHdyVZoOKnW1W7LEYkHN6-C8TTNaEvQURDFhUDyAvWLX5hUL-ssGLYJxHdCK6gqxqMOPJHcU936tSElRPQMBAu0BN4Qvm4edsCGA49Gx7JomjGjjT7U2lMUEuMALsZPKg3Sq9gaKvvMA_vRJRKKoap8XSB1Tu_8hUIu9Co1xCXgIK4_P931fq1KcPLtOT-rVnNRKLYBhVVo83-84yoIPLkg3_AxXmjyZJLlHvbiV_AHf83-Ovw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -42.8821377,
                    "lng": 147.3271949
                },
                "viewport": {
                    "northeast": {
                        "lat": -42.65537552036029,
                        "lng": 147.6133832160388
                    },
                    "southwest": {
                        "lat": -43.01412256432244,
                        "lng": 147.1336604011

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJx8ukg593J5O3V0d9IrK6BGsJazhcqB6rsLoJS3slPOXs-V3a_k8BEPTPxgqdjPgQsfbt6IdJdDz5ytzwtVb_ggn9jMJCBXDSo-HwOyrN_-zMne4cbwx4hp8DwjXsJkcDztpC7vO87oFiJAW79mD5mi_FGRO6Z0-Cfz6bz4oXRJjD4aq8EB3FGDCHKoDEVrykzyeL0hAoOyeWBD1t4TKu6DptvwJuC9YJp7mSEG9MesyUPzCj6iB05oDkfLgJ5hMX_cZrAgDq3ZgQCgO0e9MHykOlEkGILYm72AGJLt4deNWVYONQH-S8kAoBnp0iFN3EXBAJ4_NtFoOtVJElm0aGv3VDow8T8XsnnY7BkHN65HURG-NYvg3dsDHofVu6wkjyalqtFXDDmdQ4HZmjLD2UWM4SwQ2Aa8DLYHp2AcLiMz0kgpKy99g",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -34.7239963,
                    "lng": 135.8611498
                },
                "viewport": {
                    "northeast": {
                        "lat": -34.6874141941253,
                        "lng": 135.8943714047651
                    },
                    "southwest": {
                        "lat": -34.77006168425965,
                        "lng": 135.81060557274

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLP7uU5pVAuCkWoD0PK5fkOnoqpMlhy7ca635uXM4iBIInytkE_zlKp4Y0vpPGehSfp5wH7hxOlc7I0LT8hahhbV64vq98fouNrJ8JJ1CFW3gXVmdPrWf-9A3g4pSaMh0rmudzUcoF8Zo222NjAoc2jOaP47iXGDJQIK3YYtwFGP580XwSVzMvnXBeN_9Jk7diVsUjnzDn8CGzPDvKpM6cZtnw31H5PuztXMXftSUHasb8YtH6tJjioowthZYDKIXgUqbkRspTz6fTz2cXT7jbeQc4303H1nFmnb-Fv8Meuzz3QHnncm86i05b0R_pTPzbscBKdEWPyKnUt-Ik163_DlsXy-Yy2esF9GUIVmDO4SGle5EtTCZ9U4rQ0NfGJAGJ_9amwnuQhI88VEUpa0dumITxjR4wr1Z7w9k545armPx8pq7-Iqg",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 20.5072923,
                    "lng": -13.052978
                },
                "viewport": {
                    "northeast": {
                        "lat": 20.5308574421646,
                        "lng": -13.02845474322203
                    },
                    "southwest": {
                        "lat": 20.47438075238246,
                        "lng": -13.07364456045605

{
    "html_attributions": [],
    "next_page_token": "ATtYBwI4_oYeQyWXLTe_y7a8vqJ4pIwJRbq0oD4HKozhcZAigcQ7XlwZkdfKc6XZiHxKwiO0CGuV1BhbVbSiyUcqXbp0rBPjYe6QxfyZad5kpt6Q8ILvRwEYRjAIjkQoDGTIrTjFEslvJnbK69_I-X9t0cqmAQbu4jeK6BT7nopmm-7OhjUY59eAzgUU0azuasX1nAsk4uxgWKDwFBzxp82bUqZE6XWcHLDPEFEloftnkcFyRk6mUsrvUJYEbDDqU-pHJRWdVNAYePofS8CEjjVhDsYnsVuVAsqByx1XrYNJSEYQlnnBctNIp1gunPzS60fA5jyut1ckLZoxFdiAsDzGqqWTtXqTT1qN9AJ4Rs4y2SQnXNqh5-ho54ibj-p_wEgJ72QlNaVv0-tT6MMHUKMT4y3zqCUXA_PG7QshXMBjlJwgg8adZVv5Hw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -26.9584048,
                    "lng": 24.7298615
                },
                "viewport": {
                    "northeast": {
                        "lat": -26.89423200102485,
                        "lng": 24.77035292291059
                    },
                    "southwest": {
                        "lat": -27.00169406950921,
                        "lng": 24.656522980352

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKAPx_jDdjDFTaEvcmV5ToTkjx81Qg_y2pAHWU7OKCL4LKX2Bf5-yoF2MncV0Rlf1kDWmdbRAFzeeasO7k4J6Ev2nZtAdLmPI08fKX7TNjIp16GxdkjLcIVTnwMpXfb9ba8YA5hKMVw8QAt1-iWGcciS2AGUsaE-9_CAaP3S0gFfYOQ-YX0fe5cl-CDKgCcSntvkqs1e5Eivz8-K0PZ_jqh88NzuCy0v5cWMCJHoefwycDRB9Hamf3Ejr-7CiovRsvhg2NHBerMpZz7V-QEJHNOckHivBokiIJZAohSt8VTTz0SfWOGKDuKsDFm5NMseSWGOb83hir6F1chAmc4BezjsFU2_wosmEwzSElbgntWRKmwbA8_iD24amC3owiK49r_WWLAG8TcCp-B8ggkz4t6WLLUbVL4LDYk04uhD72Wwqmu2nMZJw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 31.8974232,
                    "lng": 54.3568562
                },
                "viewport": {
                    "northeast": {
                        "lat": 31.96541574262561,
                        "lng": 54.45304873906427
                    },
                    "southwest": {
                        "lat": 31.79370107931908,
                        "lng": 54.22048563595844


In [8]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,17,maltahohe,-24.8333,16.9833,73.02,16,0,4.68,NaN,1616052198,Maltahöhe Hotel
1,60,new norfolk,-42.7826,147.0587,75.99,41,0,1.99,AU,1616052231,St Ives Hobart Accommodation Tasmania
2,97,port lincoln,-34.7333,135.8667,74.75,57,0,9.66,AU,1616052258,Port Lincoln Hotel
3,288,atar,20.5169,-13.0499,70.86,13,0,8.63,MR,1616052401,Hotel Lemina Mint Maata
4,295,vryburg,-26.9566,24.7284,72.66,58,0,7.72,ZA,1616052406,Liquor Warehouse - Vryburg
5,433,ardakan,32.3100,54.0175,78.80,24,0,9.22,IR,1616052507,Mellat Bank


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))